In [ ]:
def find_lr(device, train_loader, loss_fn, init_value = 1e-2, final_value=1e3, beta=0.98, mult=1.1):
    lr = init_value
    avg_loss = 0.
    best_loss = 0.
    losses = []
    log_lrs = []
    num = int(np.log(final_value / init_value) / np.log(mult))+1
    for i in range(1,num+1):
        print(i, num)
        encoded_space_dim = 256
        encoder = Encoder(encoded_space_dim)
        decoder = Decoder(encoded_space_dim)
        params_to_optimize = [
            {'params': encoder.parameters()},
            {'params': decoder.parameters()}
        ]

        # Move both the encoder and the decoder to the selected device
        encoder.to(device)
        decoder.to(device)
        optimizer = optim.Adam(params_to_optimize, lr=lr)
    
        loss = train_epoch(encoder, decoder, device, train_loader, loss_fn, optimizer)
        avg_loss = beta * avg_loss + (1-beta) *loss
        smoothed_loss = avg_loss / (1 - beta**i)
        #Stop if the loss is exploding
        if i > 1 and smoothed_loss > 4 * best_loss:
            return log_lrs, losses
        #Record the best loss
        if smoothed_loss < best_loss or i==1:
            best_loss = smoothed_loss
        #Store the values
        losses.append(loss)
        log_lrs.append(np.log10(lr))
        #Update the lr for the next step
        lr *= mult
        optimizer.param_groups[0]['lr'] = lr
    return log_lrs, loss

# logs,losses = find_lr(device, train_loader, loss_fn)
# plt.figure(figsize=(10,8))
# plt.plot(logs,losses)
# print(logs, losses)

In [ ]:
model = FOEConvNet(args.patch_size, n_classes).to(device)
# criterion = nn.CrossEntropyLoss().to(device)
criterion = torch.nn.MSELoss().to(device)
# criterion = Orientation_Loss()

optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                                           [int(0.25 * n_epochs),
                                            int(0.50 * n_epochs),
                                            int(0.75 * n_epochs)],
                                           gamma=0.1)



class Orientation_Loss(torch.nn.Module):
    def __init__(self):
        super(Orientation_Loss,self).__init__()
        
    def forward(self, gt_in_radians, est_in_radians):
        deltas = gt_in_radians - est_in_radians
        deltas[deltas > np.pi/2.0] = np.pi - deltas[deltas > np.pi/2.0]
        delta_sqr = deltas ** 2
        totloss = torch.mean(delta_sqr)
        return totloss

# run training + validation for each epoch
train_res = []
val_res = []
for e in range(n_epochs):
    train_loss, train_err_sqr = train_model(model, train_loader, optimizer, criterion, device)
    train_rmse = np.sqrt(train_err_sqr / n_train) * 180.0 / np.pi  # in degrees
    train_res.append([train_loss, train_rmse])

    val_loss, val_err_sqr = validate_model(model, val_loader, criterion, device)
    val_rmse = np.sqrt(val_err_sqr / n_val) * 180.0 / np.pi  # in degrees
    val_res.append([val_loss, val_rmse])

    print('Epoch {}/{}: train loss / rmse = {:.4f} / {:.1f}° validation loss / rmse = {:.4f} / {:.1f}°'
          .format(e+1, n_epochs, train_loss, train_rmse, val_loss, val_rmse))
    
    scheduler.step()

    if e % 100 == 99:
        model_path = model_dir.joinpath('foe_conv_net_c{}_b{}_e{:04d}.pt'
                                        .format(n_classes, batch_size, n_epochs))
        torch.save(model.state_dict(), model_path)
        print('Saved model to {}'.format(model_path))

# save model also at the end
model_path = model_dir.joinpath('foe_conv_net_c{}_b{}_e{:04d}.pt'
                                .format(n_classes, batch_size, n_epochs))

torch.save(model.state_dict(), model_path)